### Compare to the results from Royden_Husson_06_gji

In [ ]:
import os, sys
import numpy as np
from matplotlib import pyplot as plt

# directory to the aspect Lab
ASPECT_LAB_DIR = os.environ['ASPECT_LAB_DIR']
RESULT_DIR = os.path.join(ASPECT_LAB_DIR, 'results')

sys.path.append(os.path.join(ASPECT_LAB_DIR))

import shilofue.ThDSubduction0.RoydenHusson06 as RH06

In [ ]:
yr = 365.0 * 24.0 * 3600.0
cm_per_yr = 1e-2 / yr

Vt = 1.0 * cm_per_yr
Vm = 1.0 * cm_per_yr
Vr = 1.0 * cm_per_yr
L = 1000e3
lbd = 700e3
mu = 1e20

Hele_Shaw = RH06.HELE_SHAW(Vt, Vm, L, lbd, mu)

x = 10e3 # 10 km
ym = L
N = 1000
ys = np.linspace(0, ym, N)
Vxdzs = np.zeros(N)
for i in range(N):
    y = ys[i]
    Vxdzs[i] = Hele_Shaw.Vxdz(x, y)

fig, ax = plt.subplots()
ax.plot(ys / (L / 2.0), Vxdzs / lbd / Vr)